# TECHIN 509: Melody Generator with Files & Testing

In this assignment, you will:
* Save and load melodies from files so your work is reproducible.
* Write simple tests to check that your functions behave correctly.
* Strengthen habits of writing **reliable, reusable code**.

#### Part 1 — Work with Files
**Load dataset from a file**

In [ ]:
import random
from collections import defaultdict, Counter


def load_melodies(path: str) -> list[list[tuple[str, float]]]:
    """Load melodies from a file, returning a list of (note, duration) tuples"""
    melodies = []
    try:
        with open(path, 'r') as f:
            for line in f:
                notes = []
                for token in line.strip().split():
                    if "_" in token:
                        pitch, duration = token.split("_")
                        notes.append((pitch, float(duration)))
                    else:
                        notes.append((token, 1.0))  # default duration
                melodies.append(notes)
    except FileNotFoundError:
        print(f"File not found: {path}")
        print("Please make sure the dataset file exists.")
    return melodies

def save_melodies(melodies: list[list[tuple[str, float]]], path: str) -> None:
    """Save melodies to a file, one melody per line"""
    with open(path, 'w') as f:
        for melody in melodies:
            line = " ".join(f"{note}_{duration}" for note, duration in melody)
            f.write(line + "\n")

print("Testing load_melodies function:")
melodies = load_melodies('data/melody.txt')
print(f"Loaded {len(melodies)} melodies")

if melodies:
    print(f"First melody has {len(melodies[0])} notes")
    print(f"First 5 notes: {melodies[0][:5]}")

if melodies:
    save_melodies(melodies[:2], 'data/test_save.txt')  # save first 2 melodies


Testing load_melodies function:
Loaded 9 melodies
First melody has 3 notes
First 5 notes: [('E4', 0.25), ('E4', 0.25), ('E4', 0.5)]


**Save generated melodies to a file**

In [9]:
def save_melodies(melodies: list[list[str]], path: str) -> None:
    """Save melodies (note strings) to a file, one melody per line"""
    try:
        with open(path, 'w') as file:
            for melody in melodies:
                line = " ".join(melody)  # no unpacking
                file.write(line + "\n")
        print(f"Saved {len(melodies)} melodies to {path}")
    except Exception as e:
        print(f"Error saving file: {e}")

test_melodies = [
    ['C4_0.25', 'D4_0.25', 'E4_0.25', 'G4_0.5'],
    ['F3_0.5', 'A3_0.25', 'C4_0.25'],
    ['G3_0.25', 'B3_0.25', 'D4_0.5', 'F4_0.25']
]

save_melodies(test_melodies, 'output/test_output.txt')


Saved 3 melodies to output/test_output.txt


**Robustness**

In [13]:
def load_melodies(path: str) -> list[list[str]]:
    """
    Read melodies from a file and return as list of note strings.
    
    Args:
        path: Path to the file containing melodies
        
    Returns:
        List of melodies, where each melody is a list of note strings
    """
    try:
        with open(path, 'r') as file:
            melodies = []
            for line in file:
                line = line.strip()  # remove whitespace
                if line:  # skip empty lines
                    notes = line.split()  # split by spaces
                    melodies.append(notes)
            return melodies

    except FileNotFoundError:
        print(f"File not found: {path}")
        print("Please make sure the dataset file exists.")
        return []

    except Exception as e:
        print(f"Error reading file: {e}")
        return []

# Test the robust function

print("Testing robust load_melodies function:")
melodies = load_melodies('data/melody.txt')
print(f"Loaded {len(melodies)} melodies")

if melodies:
    print(f"First melody has {len(melodies[0])} notes")
    print(f"First 5 notes: {melodies[0][:5]}")


Testing robust load_melodies function:
Loaded 8 melodies
First melody has 3 notes
First 5 notes: ['E4_0.25', 'E4_0.25', 'E4_0.5']


#### Part 2 - Add Testing

In [2]:
import unittest
import tempfile
import os

# Functions to test
def load_melodies(file_path: str) -> list[list[str]]:
    """Load melodies from a file, returning a list of melodies (each a list of note strings)."""
    try:
        with open(file_path, 'r') as f:
            melodies = [line.strip().split() for line in f if line.strip()]
        return melodies
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return []
    except Exception as e:
        print(f"Error reading file: {e}")
        return []

def save_melodies(melodies: list[list[str]], file_path: str) -> None:
    """Save melodies to a file, one melody per line."""
    try:
        with open(file_path, 'w') as f:
            for melody in melodies:
                f.write(' '.join(melody) + '\n')
    except Exception as e:
        print(f"Error saving file: {e}")

def parse_note_str(note_str: str) -> tuple[str, float]:
    """Parse a note string like 'C4_0.25' into (note, duration)."""
    parts = note_str.split('_')
    if len(parts) != 2:
        raise ValueError(f"Invalid note format: {note_str}")
    note, dur = parts[0], float(parts[1])
    return note, dur


# Unit tests
class TestMelodyFunctions(unittest.TestCase):

    def test_load_valid_file(self):
        with tempfile.NamedTemporaryFile(mode='w', delete=False) as f:
            f.write("C4_0.25 D4_0.25 E4_0.5\n")
            f.write("G3_0.5 A3_0.25\n")
            path = f.name
        try:
            melodies = load_melodies(path)
            self.assertEqual(len(melodies), 2)
            self.assertEqual(melodies[0], ["C4_0.25", "D4_0.25", "E4_0.5"])
            self.assertEqual(melodies[1], ["G3_0.5", "A3_0.25"])
        finally:
            os.unlink(path)

    def test_load_missing_file(self):
        result = load_melodies("no_such_file.txt")
        self.assertEqual(result, [])

    def test_load_skips_empty_lines(self):
        with tempfile.NamedTemporaryFile(mode='w', delete=False) as f:
            f.write("C4_0.25 D4_0.25\n")
            f.write("\n")
            f.write("E4_0.5\n")
            path = f.name
        try:
            melodies = load_melodies(path)
            self.assertEqual(len(melodies), 2)
            self.assertEqual(melodies[0], ["C4_0.25", "D4_0.25"])
            self.assertEqual(melodies[1], ["E4_0.5"])
        finally:
            os.unlink(path)

    def test_save_melodies_creates_file(self):
        melodies = [["C4_0.25", "D4_0.25"], ["G3_0.5"]]
        with tempfile.NamedTemporaryFile(delete=False) as f:
            path = f.name
        try:
            save_melodies(melodies, path)
            with open(path, 'r') as f:
                lines = f.read().splitlines()
            self.assertEqual(lines, ["C4_0.25 D4_0.25", "G3_0.5"])
        finally:
            os.unlink(path)

    def test_parse_note_valid_and_invalid(self):
        note, dur = parse_note_str("F#4_0.25")
        self.assertEqual(note, "F#4")
        self.assertAlmostEqual(dur, 0.25)

        note, dur = parse_note_str("A3_0.5")
        self.assertEqual(note, "A3")
        self.assertAlmostEqual(dur, 0.5)

        with self.assertRaises(ValueError):
            parse_note_str("C4")  # Missing duration
        with self.assertRaises(ValueError):
            parse_note_str("InvalidNoteString")

# Run tests
if __name__ == "__main__":
    unittest.main(argv=[''], exit=False, verbosity=2)




test_load_missing_file (__main__.TestMelodyFunctions.test_load_missing_file) ... ok
test_load_skips_empty_lines (__main__.TestMelodyFunctions.test_load_skips_empty_lines) ... ok
test_load_valid_file (__main__.TestMelodyFunctions.test_load_valid_file) ... ok
test_parse_note_valid_and_invalid (__main__.TestMelodyFunctions.test_parse_note_valid_and_invalid) ... ok
test_save_melodies_creates_file (__main__.TestMelodyFunctions.test_save_melodies_creates_file) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.011s

OK


File not found: no_such_file.txt
